In [1]:
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sns
import shutil
import time
from datetime import datetime
from itertools import product
from scipy.spatial.distance import cdist
from shapely import Point

In [2]:
# # DECODING PERSONALITY PROFILE QUESTIONNAIRES
# ## 1. 12-item Highly-Sensitive Person scale
# def calculate_sensitivity_scores(X):
#     '''
#     Pluess, M., Lionetti, F., Aron, E. N., & Aron, A. (2023). People differ in their sensitivity to the environment: 
#     An integrated theory, measurement and empirical evidence. Journal of Research in Personality, 104, 104377.
#     https://sensitivityresearch.com/self-tests/adult-self-test/
#     '''
#     # return {'Highly-Sensitive Person (HSP)': np.mean(X),
#     #         'Ease Of Excitation (EOE)': np.mean([X[3], X[5], X[7], X[8], X[11]]),
#     #         'Aesthetic Sensitivity (AES)': np.mean([X[0], X[2], X[4], X[9]]),
#     #         'Low Sensory Threshold (LST)': np.mean([X[1], X[6], X[10]])}
#     return (np.mean(X), # highly sensitive person (HSP) scale
#             np.mean([X[3], X[5], X[7], X[8], X[11]]), # ease of excitation (EOE)
#             np.mean([X[0], X[2], X[4], X[9]]), # aesthetic sensitivity (AES)
#             np.mean([X[1], X[6], X[10]])) # low sensory threshold (LST)

# def assign_sensitivity_group(HSP, source='paper'):
#     '''
#     Assign sensitivity group based on HSP score. There are two different cutoffs in different sources in the Sensitivity Research website:

#     1. A PAPER - Cutoffs: 3.76, 4.66
#     Lionetti, F., Aron, A., Aron, E.N., Burns, G.L., Jagiellowicz, J., & Pluess, M. (2018) Dandelions, tulips and orchids: 
#     evidence for the existence of low-sensitive, medium-sensitive and high-sensitive individuals, Translational Psychiatry 8(24).
#     https://www.doi.org/10.1038/s41398-017-0090-6

#     2. THE WEBSITE'S TEST - Cutoffs: 4, 4.625
#     https://sensitivityresearch.com/self-tests/adult-self-test/
#     '''
    
#     if source == 'paper':
#         cutoffs = [3.76, 4.66]
#     elif source == 'website':
#         cutoffs = [4, 4.625]
    
#     if HSP >= cutoffs[1]:
#         return 'high'
#     elif HSP >= cutoffs[0]:
#         return 'medium'
#     else:
#         return 'low'
    
# ## 2. Satisfaction with life scale (SWLS)
# def satisfaction_with_life(answers):
#     '''
#     Diener, E., Emmons, R. A., Larsen, R. J., & Griffin, S. (1985). The Satisfaction with Life Scale.
#     Journal of Personality Assessment, 49, 71-75.
#     https://fetzer.org/sites/default/files/images/stories/pdf/selfmeasures/SATISFACTION-SatisfactionWithLife.pdf
#     '''
#     if np.sum(answers) > 30:
#         return 7 # Extremely satisfied
#     elif np.sum(answers) > 25:
#         return 6 # Satisfied
#     elif np.sum(answers) > 20:
#         return 5 # Slightly satisfied
#     elif np.sum(answers) == 20:
#         return 4 # Neutral
#     elif np.sum(answers) >= 15:
#         return 3 # Slightly dissatisfied
#     elif np.sum(answers) >= 10:
#         return 2 # Dissatisfied
#     else:
#         return 1 # Extremely dissatisfied

# ## 3. Big Five Personality Traits via the Ten Item Personality Measure (TIPI)
# def calculate_big_five_personalities(X):
#     '''
#     Gosling, S. D., Rentfrow, P. J., & Swann Jr., W. B. (2003). A very brief measure of the Big-Five personality domains.
#     Journal of Research in Personality, 37, 504–528.
#     http://gosling.psy.utexas.edu/scales-weve-developed/ten-item-personality-measure-tipi/
#     '''
#     def reverse_score(x):
#         return 8 - x

#     return ((X[0] + reverse_score(X[5])) / 2, # extraversion
#             (reverse_score(X[1]) + X[6]) / 2, # agreeableness
#             (X[2] + reverse_score(X[7])) / 2, # conscientiousness
#             (reverse_score(X[3]) + X[8]) / 2, # emotional stability
#             (X[4] + reverse_score(X[9])) / 2) # openness to experiences

# Cozie survey and onboarding data

In [106]:
cozie_df = pd.read_csv(os.path.join(os.getcwd(), 'Cozie/results/arda_survey_results_select_en.csv'))
cozie_df.drop('Unnamed: 0', axis=1, inplace=True)

/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_67807/3218529462.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,10,15,16,17,18,19,22,23,33,35,47,72,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  cozie_df = pd.read_csv(os.path.join(os.getcwd(), 'Cozie/results/arda_survey_results_select_en.csv'))


In [105]:
# translation = {
#     'q_activity_change_other': {'네, 야외 활동': 'Yes, outdoors', '네, 식사': 'Yes, having a meal', '네, 공부': 'Yes, studying',
#                                 '네, 기타': 'Other', '네, 근무': 'Yes, working', '네, 여가': 'Yes, leisure', '아니요':  'No'},
#     'q_location_other': {'캠퍼스': 'On campus', '캠퍼스 밖': 'Off campus', '집': 'Home', '기타': 'Other'},
#     'q_location': {'실내': 'Indoors', '실외': 'Outdoors'},
#     'q_activity_indoors': {'공부한다': 'Studying', '일한다': 'Working', '기타': 'Other', '식사한다': 'Having a meal', '여가를 보낸다': 'Leisure'},
#     'q_activity_outdoors':{'기타': 'Other', '교통수단을 이용한다': 'Taking transportation', '어디론가 걸어간다': 'Walking somewhere',
#                            '밖에 앉아 있다': 'Sitting outside', '밖에서 시간을 보낸다': 'Spending time outdoors'},
#     'q_location_working': {'공용 업무공간': 'Shared workspace', '집': 'Home', '도서관': 'Library', '기타': 'Other', '카페': 'Café',
#                            '개인 업무공간': 'Private workspace'},
#     'q_location_studying': {'교실': 'Classroom', '도서관': 'Library', '기타': 'Other', '카페': 'Café', '집': 'Home'},
#     'q_location_meal': {'캠퍼스': 'On campus', '집': 'Home', 'Off campus': '캠퍼스 밖', 'Other': '기타'},
#     'q_location_leisure': {'캠퍼스': 'On campus', '집': 'Home', '캠퍼스 밖': 'Off campus', '기타': 'Other'},
#     'q_activity_change_working': {'아니요': 'No', '네, 야외 활동': 'Yes, outdoors', '네, 식사': 'Yes, having a meal', '네, 기타': 'Yes, other',
#                                   '네, 공부': 'Yes, studying', '네, 여가': 'Yes, leisure', '기타': 'Yes, other'},
#     'q_activity_change_studying': {'아니요': 'No', '네, 야외 활동': 'Yes, outdoors', '네, 기타': 'Yes, other', '네, 식사': 'Yes, having a meal',
#                                    '네, 근무': 'Yes, working', '네, 여가': 'Yes, leisure', '기타': 'Yes, other'},
#     'q_activity_change_meal': {'네, 야외 활동': 'Yes, outdoors', '네, 기타': 'Yes, other', '네, 근무': 'Yes, working', '네, 여가': 'Yes, leisure',
#                                '아니요': 'No', '네, 공부': 'Yes, studying', '기타': 'Yes, other'},
#     'q_activity_change_leisure': {'네, 야외 활동': 'Yes, outdoors', '아니요': 'No', '네, 공부': 'Yes, studying', '네, 식사': 'Yes, having a meal',
#                                   '네, 기타': 'Yes, other', '네, 근무': 'Yes, working', '기타': 'Yes, other'},
#     'q_activity_change_outdoors': {'네, 기타': 'Yes, other', '네, 근무': 'Yes, working', '네, 식사': 'Yes, having a meal', '아니요': 'No',
#                                    '네, 공부': 'Yes, studying', '네, 여가': 'Yes, leisure', '기타': 'Yes, other'},
#     'q_location_change': {'네, 45분 이상': 'Yes, more than 45 min',  '네, 15분 미만': 'Yes, less than 15 min',
#                           '네, 45분 미만': 'Yes, less than 45 min', '아니요': 'No, I did not change location',
#                           '네, 30분 미만': 'Yes, less than 30 min'},
#     'q_adaptive_comfort': {'네, 장소': 'Yes, location', '아니요': 'No', '네, 옷': 'Yes, clothing', '네, 기타': 'Yes, other',
#                            '네, 선풍기': 'Yes, fan', '네, 온도 조절기': 'Yes, thermostat'},
#     'q_clothing': {'가볍다': 'Light', '보통이다': 'Medium', '무겁다': 'Heavy', '아주 가볍다': 'Very light', '아주 무겁다': 'Very heavy'},
#     'q_thermal': {'지금 처럼': 'No Change', '더 시원하게': 'Cooler', '더 따뜻하게': 'Warmer'},
#     'q_mood': {'긍정적이다': 'Positive', '보통이다': 'Neutral', '부정적이다': 'Negative'}}
# for c in ['q_activity_change_other', 'q_location_other']:
#     cozie_df.loc[cozie_all[c].isna() & (~cozie_all[c + '_ko'].isna()), c] = cozie_all.loc[
#         cozie_all[c].isna() & (~cozie_all[c + '_ko'].isna()), c + '_ko'].replace(translation[c])
# cozie_df.replace({'네, 기타': 'Other'}, inplace=True)
# cozie_df.to_csv(os.path.join(os.getcwd(), 'Cozie/results/arda_survey_results_select_en.csv'))

In [4]:
## THE ONBOARDING SURVEY WAS MAINLY ABOUT DEMOGRAPHICS (age, gender, time spent in Seoul, etc.), PERSONALITY AND PREFERENCES
## THIS IS PROBABLY NOT VERY RELEVANT FOR YOUR WORK
# onboarding_df = pd.read_csv(
#     os.path.join(os.getcwd(), 'Cozie/results/other_surveys/Participant Onboarding Form [ 참여자 온보딩 설문]_December 4, 2023_16.03.csv'),
#     skiprows=[0,2])
# onboarding_df.rename(columns=dict(zip(onboarding_df.columns, headers)), inplace=True)
# onboarding_df.head()

In [4]:
# headers = [
#     # formalities
#     'Start Date', 'End Date', 'Response Type', 'IP Address', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date',
#     'Response ID', 'Recipient Last Name', 'Recipient First Name', 'Recipient Email', 'External Data Reference', 'Location Latitude',
#     'Location Longitude', 'Distribution Channel', 'User Language',
#     # participant demographics
#     'Experiment tag', 'Year of birth', 'Gender', 'Height (cm)', 'Weight (kg)',
#     # experience in Seoul
#     'Years in Seoul', 'Used to the weather in Seoul', 'How satisfied with the weather in Seoul',
#     'How Comfortable do you feel in Fall in Seoul', 'Enjoy being outdoors in Seoul',
#     # activities
#     'Time spent outdoors during weekdays (hours/day)',  'Time spent outdoors during weekends (hours/day)',
#     # personality
#         # HSP
#     'Aware of subtleties in your environment', 'Easily overwhelmed by things', 'Rich/complex inner life', 
#     'Rattled when you have a lot to do', 'Deeply moved by the arts or music',
#     'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows',
#     'Unpleasant to have a lot going on at once', 'Changes in your life shake you up',
#     'Notice and enjoy delicate scents/tastes/sounds/art', 'Bothered by intense stimuli', 'Nervous when observed while performing a task',
#         # SWLS
#     'Life close to ideal', 'Excellent life conditions', 'Satisfied with life', 'Got the important things in life',
#     'Change almost nothing in life',
#         # TIPI
#     'Extroverted/enthusiastic', 'Critical/quarrelsome', 'Dependable/self-disciplined', 'Anxious/easily upset',
#     'Open to new experiences/complex', 'Reserved/quiet', 'Sympathetic/warm', 'Disorganized/careless',
#     'Calm/emotionally stable', 'Conventional/unreactive',
#     # spaces used in a work week
#     'Spaces used in a work week', 'Private office (hours)', 'Open-plan office (hours)', 'Cubicle office (hours)', 'Home desk (hours)',
#     'Classroom (hours)', 'Hot-desking office (hours)', 'Other spaces (type + hours)', 'Private office (controls)',
#     'Open-plan office (controls)', 'Cubicle office (controls)', 'Home desk (controls)', 'Classroom (controls)',
#     'Hot-desking office (controls)', 'Others (controls)', 'Other spaces (controls)']

In [5]:
# # fix incorrect participant IDs
# onboarding_df.loc[4, 'Experiment tag'] = 'arda06'
# onboarding_df['Experiment tag'] = [t.lower() for t in onboarding_df['Experiment tag']]

In [6]:
# onboarding_df.loc[0, ['Gym (hours)', 'Gym (controls)']] = [3, onboarding_df.loc[0, 'Others (controls)']]
# onboarding_df.loc[2, ['Library (hours)', 'Library (controls)']] = [50, onboarding_df.loc[2, 'Others (controls)']]
# onboarding_df.loc[3, ['Cafe (hours)', 'Cafe (controls)']] = [5, onboarding_df.loc[3, 'Others (controls)']]
# onboarding_df.loc[13, ['Gym (hours)', 'Gym (controls)']] = [10, onboarding_df.loc[13, 'Others (controls)']]
# onboarding_df.loc[16, ['Library (hours)', 'Library (controls)']] = [40, onboarding_df.loc[16, 'Others (controls)']]
# onboarding_df.loc[18, ['Lab (hours)', 'Lab (controls)']] = [np.nan, onboarding_df.loc[18, 'Others (controls)']]
# onboarding_df.loc[19, ['Cafe (hours)', 'Cafe (controls)']] = [10, onboarding_df.loc[19, 'Others (controls)']]
# onboarding_df.loc[[0, 2, 3, 13, 16, 18, 19, 20], ['Other spaces (type + hours)', 'Others (controls)', 'Other spaces (controls)']] = np.nan
# onboarding_df[['Other spaces (hours)']] = onboarding_df[['Other spaces (type + hours)']].astype(float)

In [7]:
# # drop empty/useless cols
# onboarding_df.drop(['Recipient Last Name', 'Recipient First Name', 'Recipient Email', 'External Data Reference',
#                     'Other spaces (type + hours)', 'Others (controls)', 'Other spaces (controls)'], axis=1, inplace=True)

# # drop columns
# drop_columns = [
#     'Start Date', 'End Date', 'Response Type', 'IP Address', 'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date',
#     'Response ID', 'Location Latitude', 'Location Longitude', 'Distribution Channel', 'User Language',
#     'Used to the weather in Seoul', 'Spaces used in a work week']
# onboarding_df.drop(drop_columns, axis=1, inplace=True)

In [8]:
# # Make 1-7 rankings ints again
# onboarding_df.replace({'7 (매우 그렇다)': 7, '1  (전혀 그렇지 않다)': 1}, inplace=True)
# onboarding_df[['Easily overwhelmed by things', 'Rattled when you have a lot to do', 'Deeply moved by the arts or music',
#                'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows', 
#                'Unpleasant to have a lot going on at once', 'Changes in your life shake you up',
#                'Notice and enjoy delicate scents/tastes/sounds/art', 'Bothered by intense stimuli',
#                'Nervous when observed while performing a task']] = onboarding_df[
#             ['Easily overwhelmed by things', 'Rattled when you have a lot to do', 'Deeply moved by the arts or music',
#                'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows', 
#                'Unpleasant to have a lot going on at once', 'Changes in your life shake you up',
#                'Notice and enjoy delicate scents/tastes/sounds/art', 'Bothered by intense stimuli',
#                'Nervous when observed while performing a task']].astype(int)

In [9]:
# # Make Agree/Disagree shorter strings
# onboarding_df.replace({'Strongly disagree (전혀 그렇지 않다)': 1, # 'disagree_strongly',
#                        'Disagree (그렇지 않다)': 2, # 'disagree',
#                        'Somewhat disagree (약간 그렇지 않다)': 3, # 'disagree_somewhat',
#                        'Neither agree nor disagree (보통이다)': 4, # 'neither',
#                        'Somewhat agree (약간 그렇다)': 5, # 'agree_somewhat',
#                        'Agree (그렇다0': 6, # 'agree',
#                        'Agree (그렇다)': 6, # 'agree',
#                        'Strongly agree (매우 그렇다)': 7}, # 'agree_strongly'},
#                        inplace=True)

In [10]:
# # decode personality tests
# onboarding_df[['highly_sensitive_person_HSP', 'ease_of_excitation_EOE', 'aesthetic_sensitivity_AES', 'low_sensory_threshold_LST']] = \
#     onboarding_df[['Aware of subtleties in your environment', 'Easily overwhelmed by things', 'Rich/complex inner life',
#                    'Rattled when you have a lot to do', 'Deeply moved by the arts or music',
#                    'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows',
#                    'Unpleasant to have a lot going on at once', 'Changes in your life shake you up',
#                    'Notice and enjoy delicate scents/tastes/sounds/art', 'Bothered by intense stimuli',
#                    'Nervous when observed while performing a task']].T.apply(calculate_sensitivity_scores).T
# onboarding_df['satisfaction_with_life_SWLS'] = onboarding_df[['Life close to ideal', 'Excellent life conditions', 'Satisfied with life',
#                                                               'Got the important things in life', 'Change almost nothing in life']].T.apply(
#     satisfaction_with_life)
# onboarding_df[['extraversion', 'agreeableness', 'conscientiousness', 'emotional_stability', 'openness_to_experiences']] = onboarding_df[
#     ['Extroverted/enthusiastic', 'Critical/quarrelsome', 'Dependable/self-disciplined', 'Anxious/easily upset',
#      'Open to new experiences/complex', 'Reserved/quiet', 'Sympathetic/warm', 'Disorganized/careless', 'Calm/emotionally stable',
#      'Conventional/unreactive']].T.apply(calculate_big_five_personalities).T

In [11]:
# # drop personality questions after translating them
# onboarding_df.drop(
#     ['Aware of subtleties in your environment', 'Easily overwhelmed by things', 'Rich/complex inner life', 
#      'Rattled when you have a lot to do', 'Deeply moved by the arts or music', 
#      'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows',
#      'Unpleasant to have a lot going on at once', 'Changes in your life shake you up', 'Notice and enjoy delicate scents/tastes/sounds/art',
#      'Bothered by intense stimuli', 'Nervous when observed while performing a task', 'Life close to ideal', 'Excellent life conditions',
#      'Satisfied with life', 'Got the important things in life', 'Change almost nothing in life', 'Extroverted/enthusiastic',
#      'Critical/quarrelsome', 'Dependable/self-disciplined', 'Anxious/easily upset', 'Open to new experiences/complex', 'Reserved/quiet',
#      'Sympathetic/warm', 'Disorganized/careless', 'Calm/emotionally stable', 'Conventional/unreactive'], axis=1, inplace=True)

In [12]:
# # Make controls easier to verify
# for c in [col for col in onboarding_df.columns if 'controls' in col]:
#     for i in onboarding_df[c].dropna().index:
#         for j in onboarding_df.loc[i, c].split(','):
#             onboarding_df.loc[i, c.split(' (')[0] + ' / ' + j.split(' (')[0]] = True
# onboarding_df.drop([c for c  in onboarding_df.columns if 'controls' in c], axis=1, inplace=True)
# # for c in [col for col in onboarding_df.columns if 'controls' in col]:
# #     onboarding_df.loc[onboarding_df[c].dropna().index, c] = '; '.join([j.split(' (')[0] for j in onboarding_df.loc[i, c].split(',')])


In [108]:
## AGAIN, THE "MERGE" PART WAS ABOUT MERGING THE ONBOARDING DATA, WHICH IS NOT SO RELEVANT TO YOU
# cozie_merge = cozie_df.merge(onboarding_df, left_on='id_participant', right_on='Experiment tag')
cozie_merge = cozie_df

In [109]:
cozie_merge.drop(['si_iphone_location_service_enabled', 'si_watch_location_service_enabled', 'ts_location_acquisition_method',
                  'ts_location_floor', 'ts_location_source_device', 'ws_location_acquisition_method', 'ws_location_floor',
                  'ws_location_source_device', 'wss_goal', 'wss_participation_days', 'wss_participation_time_end', 
                  'wss_participation_time_start', 'wss_reminder_enabeled', 'wss_reminder_interval'], axis=1, inplace=True)

In [110]:
cozie_merge[['ts_longitude', 'ws_longitude', 'ts_latitude', 'ws_latitude']].replace({0.0: np.nan}, inplace=True)

/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_67807/3461931069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cozie_merge[['ts_longitude', 'ws_longitude', 'ts_latitude', 'ws_latitude']].replace({0.0: np.nan}, inplace=True)


In [111]:
cozie_merge['datetime'] = pd.to_datetime(cozie_merge.timestamp)

In [21]:
# # decode personality tests
# cozie_merge[['highly_sensitive_person_HSP', 'ease_of_excitation_EOE', 'aesthetic_sensitivity_AES', 'low_sensory_threshold_LST']] = \
#     cozie_merge[['Aware of subtleties in your environment', 'Easily overwhelmed by things', 'Rich/complex inner life',
#                    'Rattled when you have a lot to do', 'Deeply moved by the arts or music',
#                    'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows',
#                    'Unpleasant to have a lot going on at once', 'Changes in your life shake you up',
#                    'Notice and enjoy delicate scents/tastes/sounds/art', 'Bothered by intense stimuli',
#                    'Nervous when observed while performing a task']].T.apply(calculate_sensitivity_scores).T
# cozie_merge['satisfaction_with_life_SWLS'] = cozie_merge[['Life close to ideal', 'Excellent life conditions', 'Satisfied with life',
#                                                               'Got the important things in life', 'Change almost nothing in life']].T.apply(
#     satisfaction_with_life)
# cozie_merge[['extraversion', 'agreeableness', 'conscientiousness', 'emotional_stability', 'openness_to_experiences']] = cozie_merge[
#     ['Extroverted/enthusiastic', 'Critical/quarrelsome', 'Dependable/self-disciplined', 'Anxious/easily upset',
#      'Open to new experiences/complex', 'Reserved/quiet', 'Sympathetic/warm', 'Disorganized/careless', 'Calm/emotionally stable',
#      'Conventional/unreactive']].T.apply(calculate_big_five_personalities).T

# # drop personality questions after translating them
# cozie_merge.drop(
#     ['Aware of subtleties in your environment', 'Easily overwhelmed by things', 'Rich/complex inner life', 
#      'Rattled when you have a lot to do', 'Deeply moved by the arts or music', 
#      'Annoyed when people try to get you to do too many things at once', 'Avoid violent movies and TV shows',
#      'Unpleasant to have a lot going on at once', 'Changes in your life shake you up', 'Notice and enjoy delicate scents/tastes/sounds/art',
#      'Bothered by intense stimuli', 'Nervous when observed while performing a task', 'Life close to ideal', 'Excellent life conditions',
#      'Satisfied with life', 'Got the important things in life', 'Change almost nothing in life', 'Extroverted/enthusiastic',
#      'Critical/quarrelsome', 'Dependable/self-disciplined', 'Anxious/easily upset', 'Open to new experiences/complex', 'Reserved/quiet',
#      'Sympathetic/warm', 'Disorganized/careless', 'Calm/emotionally stable', 'Conventional/unreactive'], axis=1, inplace=True)

In [112]:
cozie_merge.drop(cozie_merge.loc[cozie_merge.wss_title == 'Weather (short)'].index, inplace=True)

In [113]:
cozie_merge = cozie_merge.loc[cozie_merge[
    ['location_change', 'q_activity_change_leisure', 'q_activity_change_meal', 'q_activity_change_other', 'q_activity_change_outdoors',
     'q_activity_change_studying', 'q_activity_change_working', 'q_activity_indoors', 'q_activity_outdoors', 'q_adaptive_comfort',
     'q_clothing', 'q_location', 'q_location_change', 'q_location_leisure', 'q_location_meal', 'q_location_other', 'q_location_studying',
     'q_location_working', 'q_mood', 'q_thermal', 'ts_HRV', 'ts_altitude', 'ts_audio_exposure_environment', 'ts_audio_exposure_headphones',
     'ts_heart_rate', 'ts_latitude', 'ts_location_accuracy_horizontal', 'ts_location_accuracy_vertical', 'ts_longitude',
     'ts_oxygen_saturation', 'ts_resting_heart_rate', 'ts_sleep_REM', 'ts_sleep_awake', 'ts_sleep_core', 'ts_sleep_deep', 'ts_sleep_in_bed',
     'ts_sleep_unspecified', 'ts_stand_time', 'ts_step_count', 'ts_timestamp_location', 'ts_walking_distance', 'ws_HRV', 'ws_altitude',
     'ws_audio_exposure_environment', 'ws_audio_exposure_headphones', 'ws_heart_rate', 'ws_latitude', 'ws_location_accuracy_horizontal',
     'ws_location_accuracy_vertical', 'ws_longitude', 'ws_oxygen_saturation', 'ws_resting_heart_rate', 'ws_sleep_REM', 'ws_sleep_awake',
     'ws_sleep_core', 'ws_sleep_deep', 'ws_sleep_in_bed', 'ws_sleep_unspecified', 'ws_stand_time', 'ws_step_count', 'ws_survey_count',
     'ws_timestamp_location', 'ws_timestamp_start', 'ws_walking_distance']].dropna(how='all').index].reset_index(drop=True)

In [114]:
# records with no coordinates AND no Cozie survey are not really useful - I'm dropping them
cozie_merge = cozie_merge.loc[cozie_merge[
    ['q_activity_change_leisure', 'q_activity_change_meal', 'q_activity_change_other', 'q_activity_change_outdoors',
     'q_activity_change_studying', 'q_activity_change_working', 'q_activity_indoors', 'q_activity_outdoors', 'q_adaptive_comfort',
     'q_clothing', 'q_location', 'q_location_change', 'q_location_leisure', 'q_location_meal', 'q_location_other', 'q_location_studying',
     'q_location_working', 'q_mood', 'q_thermal', 'ts_latitude', 'ts_longitude', 'ws_latitude', 'ws_longitude']].dropna(how='all').index] \
    .reset_index(drop=True)

# Weather data

In [115]:
sdot_locations = pd.read_csv(r'/Users/mmosteiroromer/Documents/Korea/Urban data/Weather_S-DOT/s-dot-locations.csv', encoding='cp949')

In [116]:
sdot_locations['coords'] = [(x,y) for x, y in zip(sdot_locations['Xlon'], sdot_locations['Ylat'])]
sdot_locations = gpd.GeoDataFrame(sdot_locations, geometry=[Point(x, y) for (x,y) in sdot_locations[['Xlon', 'Ylat']].values])

In [117]:
# weather_data = pd.concat([pd.read_csv(os.path.join(os.getcwd(), 'Urban data/Weather_S-DOT', f), encoding='cp949')
#            for f in os.listdir(os.path.join(os.getcwd(), 'Urban data/Weather_S-DOT'))
#                           if ('location' not in f) & ('_en' not in f) & (f[0] != '.')])

In [118]:
# weather_data.dropna(how='all', axis=1, inplace=True)
# weather_data.shape

In [30]:
# weather_data.rename(columns=dict(zip(
#     ['모델번호', '시리얼', '측정시간', '지역', '자치구', '행정동',
#      '초미세먼지 최대(㎍/㎥)', '초미세먼지 평균(㎍/㎥)', '초미세먼지 최소(㎍/㎥)', '미세먼지 최대(㎍/㎥)', '미세먼지 평균(㎍/㎥)', '미세먼지 최소(㎍/㎥)',
#      '온도 최대(℃)', '온도 평균(℃)', '온도 최소(℃)', '습도 최대(%)', '습도 평균(%)', '습도 최소(%)',
#      '풍속 최대(m/s)', '풍속 평균(m/s)', '풍속 최소(m/s)', '풍향 최대(m/s)', '풍향 평균(m/s)', '풍향 최소(m/s)',
#      '조도 최대(lux)', '조도 평균(lux)', '조도 최소(lux)', '자외선 최대(UV)', '자외선 평균(UV)', '자외선 최대(UV).1',
#      '소음 최대(dB)', '소음 평균(dB)', '소음 최소(dB)',
#      '진동(x) 최대(mm/s)', '진동(x) 평균(mm/s)', '진동(x) 최소(mm/s)', '진동(y) 최대(mm/s)', '진동(y) 평균(mm/s)', '진동(y) 최소(mm/s)',
#      '진동(z) 최대(mm/s)', '진동(z) 평균(mm/s)', '진동(z) 최소(mm/s)',
#      # '흑구온도 최대(℃)', '흑구온도 평균(℃)', '흑구온도 최소(℃)',
#      '이산화질소 최대(ppm)', '이산화질소 평균(ppm)', '이산화질소 최소(ppm)', '일산화탄소 최대(ppm)', '일산화탄소 평균(ppm)', '일산화탄소 최소(ppm)',
#      '이산화황 최대(ppm)', '이산화황 평균(ppm)', '이산화황 최소(ppm)', '암모니아 최대(ppm)', '암모니아 평균(ppm)', '암모니아 최소(ppm)', '황화수소 최대(ppm)',
#      '황화수소 평균(ppm)', '황화수소 최소(ppm)', '오존 최대(ppm)', '오존 평균(ppm)', '오존 최소(ppm)', '등록일시'],
#     # ['Model number', 'Serial', 'Time', 'Region', 'Gu', 'Dong',
#     #  'Maximum ultrafine dust (㎍/㎥)', 'Average ultrafine dust (㎍/㎥)', 'Minimum ultrafine dust (㎍/㎥)',
#     #  'Maximum fine dust (㎍/㎥)', 'Average fine dust (㎍/㎥)', 'Minimum fine dust (㎍/㎥)',
#     #  'Maximum temperature (℃)', 'Average temperature (℃)', 'Minimum temperature (℃)',
#     #  'Maximum humidity (%)', 'Average humidity (%)', 'Minimum humidity (%)', 
#     #  'Maximum wind speed (m/s)', 'Average wind speed (m/s)', 'Minimum wind speed (m/s)',
#     #  'Maximum wind direction (m/s)', 'Average wind direction (m/s)', 'Minimum wind direction (m/s)',
#     #  'Maximum illuminance (lux)', 'Average illuminance (lux)', 'Minimum illuminance (lux)',
#     #  'Maximum ultraviolet rays (UV)', 'Average ultraviolet rays (UV)', 'Maximum ultraviolet rays (UV).1',
#     #  'Maximum noise (dB)', 'Noise average (dB)', 'Noise minimum (dB)',
#     #  'Vibration (x) maximum (mm/s)', 'Vibration (x) average (mm/s)', 'Vibration (x) minimum (mm/s)',
#     #  'Vibration (y) maximum (mm/s)', 'Vibration (y) average (mm/s)', 'Vibration (y) minimum (mm/s) ',
#     #  'Vibration (z) maximum (mm/s)', 'Vibration (z) average (mm/s)', 'Vibration (z) minimum (mm/s)',
#     #  # 'Black bulb temperature maximum (℃)', 'Average dark bulb temperature (℃)', 'Minimum dark bulb temperature (℃)',
#     #  'Maximum nitrogen dioxide (ppm)', 'Average nitrogen dioxide (ppm)', 'Minimum nitrogen dioxide (ppm)',
#     #  'Maximum carbon monoxide (ppm)', 'Average carbon monoxide (ppm)', 'Minimum carbon monoxide (ppm)',
#     #  'Maximum sulfur dioxide (ppm)', 'Average sulfur dioxide (ppm)', 'Minimum sulfur dioxide (ppm)',
#     #  'Maximum ammonia (ppm)', 'Ammonia average (ppm)', 'Ammonia minimum (ppm)',
#     #  'Hydrogen sulfide maximum (ppm)', 'Hydrogen sulfide average (ppm)', 'Minimum hydrogen sulfide (ppm)',
#     #  'Maximum ozone (ppm)', 'Average ozone (ppm)', 'Minimum ozone (ppm)', 'Date of registration'],
#     ['model', 'serial', 'time_measurement', 'region', 'gu', 'dong',
#      'PM2.5_max_microgm3', 'PM2.5_mean_microgm3', 'PM2.5_min_microgm3',
#      'PM10_max_microgm3', 'PM10_mean_microgm3', 'PM10_min_microgm3',
#      'temperature_max_C', 'temperature_mean_C', 'temperature_min_C',
#      'humidity_max_pc', 'humidity_mean_pc', 'humidity_min_pc', 
#      'wind_speed_max_ms', 'wind_speed_mean_ms', 'wind_speed_min_ms',
#      'wind_direction_max_ms', 'wind_direction_mean_ms', 'wind_direction_min_ms',
#      'illuminance_max_lux', 'illuminance_mean_lux', 'illuminance_min_lux',
#      'uv_rays_max_UV', 'uv_rays_mean_UV', 'uv_rays_max_UV1',
#      'noise_max_dB', 'noise_mean_dB', 'noise_min_dB',
#      'vibration_x_max_mms', 'vibration_x_mean_mms', 'vibration_x_min_mms',
#      'vibration_y_max_mms', 'vibration_y_mean_mms', 'vibration_y_min_mms',
#      'vibration_z_max_mms', 'vibration_z_mean_mms', 'vibration_z_min_mms',
#      # 'temperature_dark_bulb_max_C', 'temperature_dark_bulb_mean_C', 'temperature_dark_bulb_min_C',
#      'no2_max_ppm', 'no2_mean_ppm', 'no2_min_ppm',
#      'co_max_ppm', 'co_mean_ppm', 'co_min_ppm',
#      'so2_max_ppm', 'so2_mean_ppm', 'so2_min_ppm',
#      'nh3_max_ppm', 'nh3_mean_ppm', 'nh3_min_ppm',
#      'h2s_max_ppm', 'h2s_mean_ppm', 'h2s_min_ppm',
#      'o3_max_ppm', 'o3_mean_ppm', 'o3_min_ppm', 'time_registration'])), inplace=True)

In [31]:
# weather_data['datetime'] = pd.to_datetime(weather_data.time_registration).dt.tz_localize('Asia/Seoul')

In [32]:
# weather_data.sort_values('datetime', inplace=True)
# weather_data.reset_index(drop=True, inplace=True)

In [119]:
base_dir = 'D:/Desktop/MSc Thesis/'
weather_data = pd.read_csv(os.path.join(base_dir, 
    'Data/S-DoT_sensors/S-DoT_NATURE_20230724-20230826_en_preprocessed.csv')).drop(
    ['Unnamed: 0.1', 'Unnamed: 0', 'time_measurement', 'time_registration'], axis=1)

weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

In [120]:
weather_data = weather_data.loc[weather_data.serial.isin(sdot_locations.SerialNum)]
weather_data.reset_index(drop=True, inplace=True)
sdot_locations = sdot_locations.loc[sdot_locations.SerialNum.isin(weather_data.serial)]
sdot_locations.reset_index(drop=True, inplace=True)

In [121]:
weather_data.head()

datetime    model       serial            region  \
0 2023-09-25 00:07:23+09:00  SDOT001  V02Q1940231   commercial_area   
1 2023-09-25 00:07:23+09:00  SDOT001  V02Q1940307  residential_area   
2 2023-09-25 00:07:23+09:00  SDOT001  V02Q1940254  residential_area   
3 2023-09-25 00:07:23+09:00  SDOT001  V02Q1940172   roads_and_parks   
4 2023-09-25 00:07:23+09:00  SDOT001  V02Q1940436  residential_area   

            gu              dong  PM2.5_max_microgm3  PM2.5_mean_microgm3  \
0  Jungnang-gu    Sangbong2-dong                 3.0                  0.0   
1  Jungnang-gu  Myeonmok3.8-dong                 3.0                  0.0   
2  Jungnang-gu      Sinnae2-dong                 0.0                  0.0   
3  Jungnang-gu    Myeonmok4-dong                 4.0                  1.0   
4      Jung-gu     Hoehyeon-dong                 0.0                  0.0   

   PM2.5_min_microgm3  PM10_max_microgm3  ...  so2_min_ppm  nh3_max_ppm  \
0                 0.0                3.0  ...          NaN          NaN   
1                 0.0                4.0  ...          NaN          NaN   
2                 0.0                0.0  ...          NaN          NaN   
3                 0.0                4.0  ...          NaN          NaN   
4                 0.0                0.0  ...          NaN          NaN   

   nh3_mean_ppm  nh3_min_ppm  h2s_max_ppm  h2s_mean_ppm  h2s_min_ppm  \
0           NaN          NaN          NaN           NaN          NaN   
1           NaN          NaN          NaN           NaN          NaN   
2           NaN          NaN          NaN           NaN          NaN   
3           NaN          NaN          NaN           NaN          NaN   
4           NaN          NaN          NaN           NaN          NaN   

   o3_max_ppm  o3_mean_ppm  o3_min_ppm  
0         NaN          NaN         NaN  
1         NaN          NaN         NaN  
2         NaN          NaN         NaN  
3         NaN          NaN         NaN  
4         NaN          NaN         NaN  

[5 rows x 60 columns]

In [131]:
for i, serial in enumerate(weather_data.serial.unique()):
    if i % 100 == 0:
        print(i)
    weather_data.loc[weather_data.serial == serial, weather_data.columns[6:]].interpolate(limit=1, inplace=True)

0
100


KeyboardInterrupt: 

In [ ]:
# The wind speed values are missing for most stations, so I'm adding a city-scale mean wind speed value for the correlation
wind_speed_city = weather_data[['datetime', 'serial', 'wind_speed_min_ms', 'wind_speed_mean_ms', 'wind_speed_max_ms']]
# Three weather stations appear to have a unit conversion error in their wind data
wind_speed_city.loc[wind_speed_city.serial.isin(['V02Q1940079', 'V02Q1940239', 'V02Q1940587']),
                    ['wind_speed_min_ms', 'wind_speed_mean_ms', 'wind_speed_max_ms']] /= 10
wind_speed_city = wind_speed_city.set_index('datetime').resample('H').mean()
wind_speed_city.rename(columns=dict(zip(
    wind_speed_city.columns, ['_'.join(c.split('_')[:2] + ['city'] + c.split('_')[2:]) for c in wind_speed_city.columns])), inplace=True)
for i in weather_data.index:
    weather_data.loc[i, wind_speed_city.columns] = wind_speed_city.loc[datetime(
        weather_data.datetime[i].year, weather_data.datetime[i].month, weather_data.datetime[i].day,
        weather_data.datetime[i].hour, 0)].values

# Urban metrics

In [ ]:
urban_metrics_sdot = pd.read_csv(r'/Users/mmosteiroromer/Documents/Korea/Urban data/Shapefile/urban_metrics_sdot.csv',
                                 index_col=0)

# Join Cozie and weather station data

In [ ]:
## find closest weather station for each point

In [ ]:
# the maximum valid distance to a weather station is defined as the maximum distance between any two weather stations in Seoul
max_distance = max([cdist([(x, y)], sdot_locations[['Xlon', 'Ylat']].values).max() for x,y in sdot_locations['coords'].values])

In [ ]:
# # Create a GeoDataFrame with Points corresponding to the location for each datapoint
# cozie_gdf = gpd.GeoDataFrame(
#     cozie_merge,
#     geometry=[Point(cozie_merge.loc[i, 'ts_longitude'], cozie_merge.loc[i, 'ts_latitude']) if not np.isnan(cozie_merge.loc[i, 'ts_latitude'])
#               else Point(cozie_merge.loc[i, 'ws_longitude'], cozie_merge.loc[i, 'ws_latitude']) for i in cozie_merge.index])
# cozie_gdf[weather_data.columns[2:]] = None

In [ ]:
# import time
# t0 = time.time()

# for x, i in enumerate(cozie_gdf.geometry.dropna().index):
#     if x % 1000 == 0:
#         print(x, time.time() - t0)
#     dist = sdot_gdf.loc[sdot_gdf.SerialNum.isin(
#         weather_data.loc[
#             (weather_data.datetime == weather_data.loc[(weather_data['datetime'] - cozie_gdf['datetime'][i]).abs().sort_values().index[0],
#              'datetime']) & (~weather_data.temperature_mean_C.isna()), 'serial'].values)].distance(cozie_gdf.geometry[i]).sort_values()
#     # only consider data points that are no further away than the maximum distance
#     if len(dist.loc[dist < max_distance]) > 0:
#         # get the weather data for the closest weather station
#         df = weather_data.loc[weather_data.serial == sdot_gdf.loc[dist.loc[dist < max_distance].index[0], 'SerialNum']].set_index('datetime')
#         # drop any duplicate indeces
#         df = df.reset_index().drop(df.reset_index().loc[df.reset_index().datetime.duplicated()].index).set_index('datetime')
#         # assign this station's data to the current datapoint
#         cozie_gdf.loc[i, weather_data.columns[2:]] = df.loc[df.index[df.index.get_indexer([cozie_gdf['datetime'][i]], method='nearest')],
#                                                             df.columns[1:]].values[0]
# print(x, time.time() - t0)

In [56]:
sdot_locations.set_index('SerialNum', inplace=True)

In [64]:
cozie_merge['latitude'] = [cozie_merge.loc[i, 'ts_latitude'] if not np.isnan(cozie_merge.loc[i, 'ts_latitude'])
                           else cozie_merge.loc[i, 'ws_latitude'] for i in cozie_merge.index]
cozie_merge['longitude'] = [cozie_merge.loc[i, 'ts_longitude'] if not np.isnan(cozie_merge.loc[i, 'ts_latitude'])
                            else cozie_merge.loc[i, 'ws_longitude'] for i in cozie_merge.index]

import time
t0 = time.time()

for x, i in enumerate(cozie_merge.index):
    if x % 1000 == 0:
        print(x, time.time() - t0)
    dist = np.sqrt((sdot_locations.Xlon - cozie_merge.loc[i, 'longitude']) ** 2 +
                   (sdot_locations.Ylat - cozie_merge.loc[i, 'latitude']) ** 2).sort_values()
    # only consider data points that are no further away than the maximum distance
    if len(dist.loc[dist < max_distance]) > 0:
        closest_sdot = dist.index[0]
        cozie_merge.loc[i, 'closest_sdot'] = closest_sdot
        # get the weather data for the closest weather station
        df = weather_data.loc[weather_data.serial == closest_sdot].set_index('datetime')
        # drop any duplicate indeces
        df = df.reset_index().drop(df.reset_index().loc[df.reset_index().datetime.duplicated()].index).set_index('datetime')
        if not np.isnan(df.loc[df.index[df.index.get_indexer([cozie_merge['datetime'][i]], method='nearest')],
                       'temperature_mean_C'].values[0]):
            # if not NaN, assign this station's data to the current datapoint
            cozie_merge.loc[i, weather_data.columns[2:]] = df.loc[df.index[df.index.get_indexer([cozie_merge['datetime'][i]],
                                                                                                method='nearest')], df.columns[1:]].values[0]
        else:
            # find closest weather station with data
            found = False
            n = 0
            while not found:
                n += 1
                next_closest_sdot = dist.index[n]
                # get the weather data for the closest weather station
                df = weather_data.loc[weather_data.serial == next_closest_sdot].set_index('datetime')
                # drop any duplicate indeces
                df = df.reset_index().drop(df.reset_index().loc[df.reset_index().datetime.duplicated()].index).set_index('datetime')
                if not np.isnan(df.loc[df.index[df.index.get_indexer([cozie_merge['datetime'][i]], method='nearest')],
                               'temperature_mean_C'].values[0]):
                    # assign this station's data to the current datapoint
                    cozie_merge.loc[i, weather_data.columns[2:]] = df.loc[df.index[df.index.get_indexer(
                        [cozie_merge['datetime'][i]], method='nearest')], df.columns[1:]].values[0]
                    found = True                

0 0.0013802051544189453
1000 98.9066162109375
2000 189.41142511367798
3000 273.935574054718
4000 365.45814514160156
5000 463.4753541946411
6000 537.753356218338
7000 587.3192193508148
8000 676.2148354053497
9000 759.8653030395508
10000 849.1474092006683
11000 921.2144560813904
12000 1043.6320292949677
13000 1150.7820191383362
14000 1247.8233082294464
15000 1324.137008190155
16000 1401.7803542613983
17000 1495.608929157257
18000 1589.0370271205902
19000 1684.3108463287354
20000 1775.9624662399292
21000 1876.9214951992035
22000 1983.1979830265045
23000 2067.3185091018677
24000 2195.0817782878876
25000 2311.4689371585846
26000 2421.4498522281647
27000 6305.437730073929
28000 9601.702684164047
29000 11755.487632274628
30000 11874.18645119667
31000 11996.802090168
32000 12141.036249160767
33000 12236.84994506836
34000 12339.72405719757
35000 12391.599863052368
36000 14402.058001279831
37000 14548.511587142944
38000 14688.05038022995
39000 14812.467330217361
40000 14889.65353512764
41000 149

In [93]:
cozie_merge[['latitude', 'longitude']] = cozie_merge[['latitude', 'longitude']].replace({0.0: np.nan})

In [155]:
# cozie_gdf.rename(columns={'serial': 'closest_sdot'}, inplace=True)
cozie_merge.drop('serial', axis=1, inplace=True)

In [156]:
# cozie_gdf[[c for c in cozie_gdf.columns if np.any(['_min_' in c, '_max_' in c, '_mean_' in c])]] = cozie_gdf[
#     [c for c in cozie_gdf.columns if np.any(['_min_' in c, '_max_' in c, '_mean_' in c])]].astype(float)
cozie_merge[[c for c in cozie_merge.columns if np.any(['_min_' in c, '_max_' in c, '_mean_' in c])]] = cozie_merge[
    [c for c in cozie_merge.columns if np.any(['_min_' in c, '_max_' in c, '_mean_' in c])]].astype(float)

In [158]:
# cozie_merge = cozie_gdf[[c for c in cozie_gdf.columns if np.all(['_max_' not in c, '_min_' not in c])]].drop('geometry', axis=1)
cozie_merge = cozie_merge[[c for c in cozie_merge.columns if np.all(['_max_' not in c, '_min_' not in c])]]
cozie_merge.head()

id_participant location_change q_activity_change_leisure  \
0         arda01            True                       NaN   
1         arda01            True                       NaN   
2         arda01            True                       NaN   
3         arda01             NaN                       NaN   
4         arda01             NaN                       NaN   

  q_activity_change_meal q_activity_change_other q_activity_change_outdoors  \
0                    NaN                     NaN                        NaN   
1                    NaN                     NaN                        NaN   
2                    NaN                     NaN                        NaN   
3                    NaN                     NaN                        NaN   
4                    NaN                     NaN                        NaN   

  q_activity_change_studying q_activity_change_working q_activity_indoors  \
0                        NaN                       NaN                NaN   
1                        NaN                       NaN                NaN   
2                        NaN                       NaN                NaN   
3                         No                       NaN           Studying   
4                        NaN                        No            Working   

  q_activity_outdoors  ... vibration_x_mean_mms vibration_y_mean_mms  \
0                 NaN  ...                  NaN                  NaN   
1                 NaN  ...                  NaN                  NaN   
2                 NaN  ...                  NaN                  NaN   
3                 NaN  ...                  NaN                  NaN   
4                 NaN  ...                  NaN                  NaN   

  vibration_z_mean_mms no2_mean_ppm co_mean_ppm so2_mean_ppm nh3_mean_ppm  \
0                  NaN          NaN         NaN          NaN          NaN   
1                  NaN          NaN         NaN          NaN          NaN   
2                  NaN          NaN         NaN          NaN          NaN   
3                  NaN          NaN         NaN          NaN          NaN   
4                  NaN          NaN         NaN          NaN          NaN   

  h2s_mean_ppm o3_mean_ppm wind_speed_city_mean_ms  
0          NaN         NaN                   0.752  
1          NaN         NaN                   0.752  
2          NaN         NaN                   0.752  
3          NaN         NaN                   0.752  
4          NaN         NaN                   0.525  

[5 rows x 155 columns]

In [161]:
cozie_merge.temperature_mean_C.min(), cozie_merge.temperature_mean_C.max()

(-6.2, 30.7)

In [162]:
# merge ws and ts columns
for ts_col in [col for col in cozie_merge.columns if 'ts_' in col]:
    ws_col = '_'.join(['ws'] + ts_col.split('_')[1:])
    new_col = '_'.join(ts_col.split('_')[1:])
    cozie_merge[new_col] = cozie_merge[ts_col]
    cozie_merge.loc[cozie_merge[new_col].isna(), new_col] = cozie_merge.loc[cozie_merge[new_col].isna(), ws_col]
## NOT SURE IF THIS IS APPROPRIATE, AS IT MIGHT LEAD TO DOUBLE-COUNTING. HERE ARE THE COLUMNS THAT ARE DOUBLE COUNTED AND THOSE THAT AREN'T
# keep_ts = ['ts_HRV', 'ts_audio_exposure_environment', 'ts_audio_exposure_headphones', 'ts_heart_rate', 'ts_oxygen_saturation',
#            'ts_resting_heart_rate', 'ts_sleep_REM', 'ts_sleep_awake', 'ts_sleep_core', 'ts_sleep_deep', 'ts_sleep_in_bed',
#            'ts_sleep_unspecified', 'ts_stand_time', 'ts_step_count', 'ts_walking_distance']
# keep_both = ['ts_altitude', 'ts_latitude', 'ts_location_accuracy_horizontal', 'ts_location_accuracy_vertical', 'ts_longitude',
#              'ts_timestamp_location']

/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_40547/3512874188.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cozie_merge[new_col] = cozie_merge[ts_col]
/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_40547/3512874188.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cozie_merge.loc[cozie_merge[new_col].isna(), new_col] = cozie_merge.loc[cozie_merge[new_col].isna(), ws_col]
/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_40547/3512874188.py:5: SettingWithCopyWarning

# Join Cozie and urban metrics

In [122]:
len(cozie_merge)

54414

In [172]:
for f in ['cozie_sdot_merge_interpolate_fsi.csv', 'cozie_sdot_merge_interpolate_fsi.shp']:
    shutil.copy(os.path.join(os.getcwd(), f),
                os.path.join(os.getcwd(), f.split('.')[0] + '_v0.' + f.split('.')[1]))

In [183]:
len(cozie_merge)

54414

In [184]:
print(len(cozie_merge))
cozie_merge = cozie_merge.merge(urban_metrics_sdot.loc[cozie_merge.closest_sdot.dropna().unique()].reset_index(),
                                left_on='closest_sdot', right_on='SerialNum', how='outer')
print(len(cozie_merge))

54414
54414


In [185]:
cozie_merge.to_csv(os.path.join(os.getcwd(), 'cozie_sdot_merge_interpolate_fsi.csv'))

In [189]:
# Create a GeoDataFrame with Points corresponding to the location for each datapoint
cozie_gdf = gpd.GeoDataFrame(
    cozie_merge,
    geometry=[Point(cozie_merge.loc[i, 'longitude'], cozie_merge.loc[i, 'latitude']) for i in cozie_merge.index],
    crs="epsg:4326").to_crs('EPSG:5174')
# # cozie_gdf[weather_data.columns[2:]] = None
# cozie_gdf = cozie_gdf.merge(urban_metrics_sdot.loc[cozie_gdf.closest_sdot.dropna().unique()].reset_index(),
#                             left_on='closest_sdot', right_on='SerialNum', how='outer')

In [199]:
cozie_gdf['datetime'] = cozie_gdf.datetime.astype(str)
cozie_gdf.to_file(os.path.join(os.getcwd(), 'cozie_sdot_merge_interpolate_fsi.shp'))

/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_40547/802132154.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  cozie_gdf.to_file(os.path.join(os.getcwd(), 'cozie_sdot_merge_interpolate_fsi.shp'))


In [4]:
cozie_merge = pd.read_csv(os.path.join(os.getcwd(), 'cozie_sdot_merge_interpolate_fsi.csv'))
cozie_merge.drop('Unnamed: 0', axis=1, inplace=True)
cozie_merge['datetime'] = pd.to_datetime(cozie_merge.datetime)

/var/folders/6s/yzkpf7gd66n560910_9fjrrmv7vy_s/T/ipykernel_45249/2671242088.py:4: DtypeWarning: Columns (5,17,77,79,81,82,85,94,95,97,98,99,100,101,103,104,105,106,133) have mixed types. Specify dtype option on import or set low_memory=False.
  cozie_merge = pd.read_csv(os.path.join(os.getcwd(), 'cozie_sdot_merge_interpolate_fsi.csv'))


In [200]:
cozie_merge['temperature_mean_C'].min()

-6.2

# Redefine survey columns

In [135]:
columns_by_type = {
    'base_info': ['id_participant', 'datetime', 'latitude', 'longitude'],
    'physiological': [
        ## NOTE: THE MEASUREMENTS BELOW DID NOT HAVE AN ASSOCIATED LOCATION AND CAN THEREFORE NOT BE CORRELATED TO ONE WEATHER STATION
        # 'HRV', 'audio_exposure_environment', 'audio_exposure_headphones', 'oxygen_saturation', 'resting_heart_rate', 'stand_time',
        # 'step_count',
                      'altitude', 'heart_rate', 'walking_distance'],
    'demographics': ['Year of birth', 'Gender', 'Height (cm)', 'Weight (kg)', 'Years in Seoul'],
    'perception': ['How satisfied with the weather in Seoul', 'How Comfortable do you feel in Fall in Seoul',
                   'Enjoy being outdoors in Seoul', 'Time spent outdoors during weekdays (hours/day)',
                   'Time spent outdoors during weekends (hours/day)'],
    'personality': ['highly_sensitive_person_HSP', 'ease_of_excitation_EOE', 'aesthetic_sensitivity_AES', 'low_sensory_threshold_LST',
                    'satisfaction_with_life_SWLS', 'extraversion', 'agreeableness', 'conscientiousness', 'emotional_stability',
                    'openness_to_experiences'],
    'weather_station': ['PM2.5_mean_microgm3', 'PM10_mean_microgm3', 'temperature_mean_C', 'humidity_mean_pc', 'illuminance_mean_lux',
                        'uv_rays_mean_UV', 'noise_mean_dB', # 'wind_speed_mean_ms', 'wind_direction_mean_ms',
                        'wind_speed_city_mean_ms'],
    'surveys': ['indoors_outdoors', 'current_location', 'current_activity', 'previous_activity', 'location_change_time',
                'adaptive_comfort', 'current_clothing', 'current_mood', 'current_thermal_preference'],
    'urban_metrics': ['FSI_100', 'FSI_200', 'FSI_300', 'GSI_100', 'GSI_200', 'GSI_300', 'OSR_100', 'OSR_200', 'OSR_300']
}

In [136]:
cozie_merge.columns

Index(['id_participant', 'location_change', 'q_activity_change_leisure',
       'q_activity_change_meal', 'q_activity_change_other',
       'q_activity_change_outdoors', 'q_activity_change_studying',
       'q_activity_change_working', 'q_activity_indoors',
       'q_activity_outdoors', 'q_adaptive_comfort', 'q_clothing', 'q_location',
       'q_location_change', 'q_location_leisure', 'q_location_meal',
       'q_location_other', 'q_location_studying', 'q_location_working',
       'q_mood', 'q_thermal', 'timestamp', 'ts_HRV', 'ts_altitude',
       'ts_audio_exposure_environment', 'ts_audio_exposure_headphones',
       'ts_heart_rate', 'ts_latitude', 'ts_location_accuracy_horizontal',
       'ts_location_accuracy_vertical', 'ts_longitude', 'ts_oxygen_saturation',
       'ts_resting_heart_rate', 'ts_sleep_REM', 'ts_sleep_awake',
       'ts_sleep_core', 'ts_sleep_deep', 'ts_sleep_in_bed',
       'ts_sleep_unspecified', 'ts_stand_time', 'ts_step_count',
       'ts_timestamp_location', 'ts_

In [137]:
# create new DataFrame to contain data with selected columns
cozie_select = cozie_merge.copy()
# create 'current_activity'
cozie_select['current_activity'] = cozie_select['q_activity_indoors']
cozie_select['current_activity'].replace({'Other': 'Other indoors'}, inplace=True)
cozie_select.loc[cozie_select.q_activity_outdoors.dropna().index, 'current_activity'] = cozie_select.q_activity_outdoors.dropna()
cozie_select.current_activity.replace({'Other': 'Other outdoors'}, inplace=True)
print('current_activity', cozie_select.current_activity.unique())
# create 'previous_activity'
cozie_select['previous_activity'] = cozie_select['q_activity_change_studying']
cozie_select['previous_activity'].replace({'No': 'Studying'}, inplace=True)
cozie_select.loc[cozie_select.q_activity_change_working.dropna().index, 'previous_activity'] = cozie_select.q_activity_change_working.dropna()
cozie_select['previous_activity'].replace({'No': 'Working'}, inplace=True)
cozie_select.loc[cozie_select.q_activity_change_leisure.dropna().index, 'previous_activity'] = cozie_select.q_activity_change_leisure.dropna()
cozie_select['previous_activity'].replace({'No': 'Leisure'}, inplace=True)
cozie_select.loc[cozie_select.q_activity_change_meal.dropna().index, 'previous_activity'] = cozie_select.q_activity_change_meal.dropna()
cozie_select['previous_activity'].replace({'No': 'Having a meal'}, inplace=True)
cozie_select.loc[cozie_select.q_activity_change_other.dropna().index, 'previous_activity'] = cozie_select.q_activity_change_other.dropna()
cozie_select['previous_activity'].replace({'No': 'Other'}, inplace=True)
cozie_select.loc[
    cozie_select.q_activity_change_outdoors.dropna().index, 'previous_activity'] = cozie_select.q_activity_change_outdoors.dropna()
cozie_select['previous_activity'].replace({'No': 'Outdoors'}, inplace=True)
cozie_select['previous_activity'].replace({'Yes, studying': 'Studying', 'Yes, working': 'Working', 'Yes, leisure': 'Leisure',
                                          'Yes, having a meal': 'Having a meal', 'Yes, outdoors': 'Outdoors', '네, 기타': 'Other'},
                                         inplace=True)
print('previous_activity', cozie_select.previous_activity.unique())
# create 'current_location'
cozie_select['current_location'] = cozie_select['q_location_studying']
cozie_select['current_location'].replace({'Other': 'Other studying'}, inplace=True)
for c in ['q_location_working', 'q_location_leisure', 'q_location_meal', 'q_location_other']:
    cozie_select.loc[cozie_select[c].dropna().index, 'current_location'] = cozie_select[c].dropna()
    cozie_select['current_location'].replace({'Other': 'Other ' + c.split('_')[-1]}, inplace=True)
print('current_location', cozie_select.current_location.unique())
cozie_select['location_change_time'] = cozie_select['q_location_change']
cozie_select['indoors_outdoors'] = cozie_select['q_location']
cozie_select['adaptive_comfort'] = cozie_select['q_adaptive_comfort']
cozie_select['current_clothing'] = cozie_select['q_clothing']
cozie_select['current_mood'] = cozie_select['q_mood']
cozie_select['current_thermal_preference'] = cozie_select['q_thermal']
# select relevant columns
cozie_select = cozie_select[[c for k in columns_by_type.keys() for c in columns_by_type[k]]]

current_activity [nan 'Studying' 'Working' 'Other indoors' 'Having a meal' 'Other outdoors'
 'Leisure' 'Taking transportation' 'Walking somewhere' 'Sitting outside'
 'Spending time outdoors']
previous_activity [nan 'Studying' 'Working' 'Outdoors' 'Having a meal' 'Other' 'Leisure']
current_location [nan 'Classroom' 'Shared workspace' 'On campus' 'Library' 'Off campus'
 'Other studying' 'Home' 'Café' 'Other leisure' 'Other working'
 'Other meal' 'Other other' 'Private workspace']
